In [206]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
%matplotlib inline

## Import Train/Test

In [261]:
train_raw = pd.read_csv('./data/train.csv')

In [262]:
train_df = pd.read_pickle('train.pkl')
test_df = pd.read_pickle('test.pkl')

In [263]:
train_df= train_df.drop('SalePrice', axis=1)

In [264]:
non_dummies = [x for x in train_df.columns if not '_' in x ]
train_df[non_dummies].shape

(1460, 23)

## Remove Features

In [265]:
# train_df = train_df.drop(
#     'PoolArea', axis=1) # mostly 0 and coleniar with 3SsnPorch and there is a binary for poolquality

## Binary for 3SsnPorch

In [266]:
train_df['3SsnPorch'] = train_df['3SsnPorch'].isna().apply(lambda x: int(not x))

# Dates

In [267]:
def add_dummies_remove_modes(dummy_list, df_raw, df):
    """ 
    add_dummies_remove_modes (dummy_list, df_raw, df)
    takes a list of column names `dummy_list` to dummify then drop 
    after dummification, a reference dataframe `df_raw` to search for 
    the dominant value of each varable in `dummy_list` then drop the 
    dominant dummy variable and a dataframe `df` you wish to concat 
    dummified variables to.
    """

    dummy_modes = list(df_raw[dummy_list].mode().iloc[0,:].items())

    dummy_modes = [(col, (float(mode))) 
                   if type(mode) == int else (col, mode) for col, mode in dummy_modes]

    drop_modes = list(map(lambda x: str(x[0]) + '_' + str(x[1]), dummy_modes))

    dummy_cols = pd.get_dummies(
        df[dummy_list].astype(object), dummy_na=True).drop(drop_modes, axis=1)

    return pd.concat([
       df.drop(dummy_list, axis=1),
       dummy_cols], axis=1)


In [242]:
# dates = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt', 'MoSold', 'YrSold']
# train_df[dates].describe()

## Change remodled year to years since remodeled

In [268]:
train_df['YearRemodAdd']  = (2010 - train_df['YearRemodAdd'])

In [269]:
test_df['YearRemodAdd']  = (2010 - test_df['YearRemodAdd'])

## Bin years then dummifying

In [270]:
Yb = train_df['YearBuilt']

range_bins = [(1800,1900), (1900,1910), (1910,1920), (1920,1930), (1930,1940), (1940,1950), (1950,1960), (1960,1970), (1970,1980), (1980,1990), (1990,2000), (2000,2011)]

bins = {r:x for (x,y) in range_bins for r in range(x,y)}

train_df = pd.concat([train_df.drop('YearBuilt', axis=1), Yb.map(bins)], axis=1)

In [271]:
Yb = test_df['YearBuilt']
test_df = pd.concat([test_df.drop('YearBuilt', axis=1), Yb.map(bins)], axis=1)

## Convert Year to Age

In [272]:
# train_df[['YearBuilt']] = train_df[['YearBuilt']].apply(lambda x: x.max() - x)

In [273]:
# test_df[['YearBuilt']] = test_df[['YearBuilt']].apply(lambda x: x.max() - x)

## Dummify MoSold and YrSold

In [274]:
# train_df = add_dummies_remove_modes(['MoSold', 'YrSold'], train_df, train_df)

In [275]:
cont = train_df.T[np.array(train_df.nunique() > 80)].T.columns.delete(-1)
scale = StandardScaler()
train_df[cont] = scale.fit_transform(train_df[cont])

/home/gavagai/.conda/envs/data_science/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gavagai/.conda/envs/data_science/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype object were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [276]:
cont = test_df.T[np.array(test_df.nunique() > 80)].T.columns.delete(-1)
scale = StandardScaler()
test_df[cont] = scale.fit_transform(test_df[cont])

/home/gavagai/.conda/envs/data_science/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/gavagai/.conda/envs/data_science/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


## Reorder

In [277]:
test_df = test_df[train_df.columns]

# Add SalePrice

In [278]:
train_df['SalePrice'] = train_raw['SalePrice']#.apply(lambda x: np.log(x))

# Pickle

In [279]:
train_df.to_pickle('train_engineered.pkl')

In [280]:
test_df.to_pickle('test_engineered.pkl')